### Imports 

In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [53]:
from fastai.imports import *
from fastai.structured import *

from pandas_summary import DataFrameSummary
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from IPython.display import display

from sklearn import metrics

In [3]:
from sklearn.linear_model import LogisticRegression

In [4]:
import lightgbm as lgb

In [5]:
from scipy.stats import ttest_rel

### Const and Func

In [6]:
np.random.seed(42)

In [7]:
path = Path('data/dataset/')

In [8]:
path

PosixPath('data/dataset')

In [9]:
! ls {path}

 bs_avg_kpi.csv   '!dataset_fields_description.xlsx'   test
 bs_chnn_kpi.csv   sample-submit-zero.csv	       train


In [10]:
! ls {path/'train'}

subs_bs_consumption_train.csv	 subs_csi_train.csv
subs_bs_data_session_train.csv	 subs_features_train.csv
subs_bs_voice_session_train.csv


In [11]:
def display_all(df):
    with pd.option_context("display.max_rows", 1000, "display.max_columns", 1000): 
        display(df)

In [12]:
def margin_of_error(z, data):
    n, std = len(data), data.std()
    z_critical = scipy.stats.norm.ppf(q=z)
    return z_critical * (std / np.sqrt(n))

### Loads

In [13]:
train_ids = pd.read_csv(path/'train/subs_csi_train.csv', sep=';')

In [14]:
test_ids = pd.read_csv(path/'test/subs_csi_test.csv', sep=';')

In [15]:
features_train = pd.read_csv(path/'train/subs_features_train.csv', sep=';', decimal=',', 
                              parse_dates=['SNAP_DATE'], infer_datetime_format=True, dayfirst=True)

In [16]:
features_test = pd.read_csv(path/'test/subs_features_test.csv', sep=';', decimal=',', 
                              parse_dates=['SNAP_DATE'], infer_datetime_format=True, dayfirst=True)

In [17]:
# convert to months (int)
features_train.SNAP_DATE = features_train.SNAP_DATE.dt.month

In [18]:
# convert to months (int)
features_test.SNAP_DATE = features_test.SNAP_DATE.dt.month

In [19]:
consum_train = pd.read_csv(path/'train/subs_bs_consumption_train.csv', sep=';', decimal=',')

In [20]:
consum_test = pd.read_csv(path/'test/subs_bs_consumption_test.csv', sep=';', decimal=',')

In [21]:
consum_train['MON'] = pd.to_datetime(consum_train['MON'], format='%d.%m').dt.month
consum_test['MON'] = pd.to_datetime(consum_test['MON'], format='%d.%m').dt.month

In [23]:
consum_train.tail()

,SK_ID,CELL_LAC_ID,MON,SUM_MINUTES,SUM_DATA_MB,SUM_DATA_MIN
1092539,3090,2873,3,0.001530,0.000000,0.000000
1092540,3090,147674,4,0.000000,0.000027,0.007549
1092541,3090,242332,4,0.000000,0.000014,0.001468
1092542,3090,156910,4,0.002667,0.000115,0.011932
1092543,3090,242062,4,0.000000,0.000017,0.000467


In [24]:
consum_test.head()

,SK_ID,CELL_LAC_ID,MON,SUM_MINUTES,SUM_DATA_MB,SUM_DATA_MIN
0,6357,104453,2,0.000000,0.000000e+00,0.000234
1,6357,8217,2,0.000000,2.200000e-07,0.001167
2,6357,412480,2,0.002722,5.590000e-06,0.010187
3,6357,355470,2,0.000000,5.309000e-05,0.002480
4,6357,45770,2,0.000000,4.000000e-08,0.000084


###  Data processing

In [25]:
raw_train = features_train[features_train.SNAP_DATE.isin([6,7,8,9,10,11,12])].copy()
raw_test = features_test[features_test.SNAP_DATE.isin([1,2,3,4,5])].copy()

In [100]:
raw_consum_train = consum_train.copy()
raw_consum_test = consum_test.copy()

#### Target Encoding for CELL_LAC_ID

In [28]:
cells_to_targ_train = pd.merge(left=consum_train[['SK_ID', 'CELL_LAC_ID']],
                               right=train_ids.drop('CONTACT_DATE', axis=1), on='SK_ID', how='left')

In [30]:
cells_to_targ_train.head()

,SK_ID,CELL_LAC_ID,CSI
0,1827,240277,1
1,1827,355195,1
2,1827,238554,1
3,1827,350978,1
4,1827,315010,1


In [31]:
cell2targ_gr = cells_to_targ_train.groupby('CELL_LAC_ID', as_index=False).agg({'CSI':'mean'})

In [32]:
cell2targ_gr.rename(columns={'CSI':'cell_encoding'}, inplace=True)

In [33]:
cell2targ_gr.head()

,CELL_LAC_ID,cell_encoding
0,1,0.00
1,2,0.25
2,3,0.00
3,4,0.00
4,5,1.00


####  Aggregate consumption

In [35]:
for df in (raw_consum_train, raw_consum_test):
    df.drop(['MON', 'SK_ID'], axis=1, inplace=True)

In [37]:
raw_consum_train = pd.merge(raw_consum_train, cell2targ_gr, on='CELL_LAC_ID', how='left')

In [41]:
raw_consum_train.head()

,CELL_LAC_ID,SUM_MINUTES,SUM_DATA_MB,SUM_DATA_MIN,cell_encoding
0,240277,0.000000,0.000002,0.000234,0.142857
1,355195,0.000072,0.000000,0.000000,0.500000
2,238554,0.000000,0.000019,0.000326,0.333333
3,350978,0.016576,0.002260,0.004329,0.750000
4,315010,0.012755,0.001186,0.002491,1.000000


In [43]:
consum_gr_tr = raw_consum_train.groupby('CELL_LAC_ID', as_index=False).agg(['mean'])
consum_gr_test = raw_consum_test.groupby('CELL_LAC_ID', as_index=False).agg(['mean'])

In [44]:
for df in (consum_gr_tr, consum_gr_test):
    df.columns = ['_'.join(c) for c in df.columns]

In [46]:
consum_gr_tr.head()

,SUM_MINUTES_mean,SUM_DATA_MB_mean,SUM_DATA_MIN_mean,cell_encoding_mean
CELL_LAC_ID,,,,
1,0.000536,0.000000,0.000000,0.00
2,0.000668,0.000001,0.000277,0.25
3,0.000000,0.000027,0.000952,0.00
4,0.003102,0.000000,0.000000,0.00
5,0.000360,0.000000,0.000000,1.00


#### Train model to predict test cell encodings

In [47]:
consum_gr_tr['foldID'] = np.random.randint(0,5+1,len(consum_gr_tr))

In [49]:
# make train and validation set
all_features = consum_gr_tr.drop(['foldID', 'cell_encoding_mean'], axis=1).copy()

In [59]:
all_y = consum_gr_tr.cell_encoding_mean

In [64]:
scores = []
for i in range(5):
    # create ids for train and validation
    idx_train = consum_gr_tr.loc[consum_gr_tr.foldID != i].index
    idx_val = consum_gr_tr.loc[consum_gr_tr.foldID == i].index
    
    
    
    X_train, X_valid = all_features.loc[idx_train], all_features.loc[idx_val]
    y_train, y_valid = all_y[idx_train], all_y[idx_val]
    
    m = RandomForestRegressor(n_estimators=100, n_jobs=-1)
    m.fit(X_train, y_train)
    
    y_pred = m.predict(X_valid)
    scores.append(metrics.mean_squared_error(y_valid, y_pred))
    
print(np.sqrt(scores))
    

[0.33532 0.33567 0.33585 0.33685 0.33682]


In [65]:
m = RandomForestRegressor(n_estimators=100, n_jobs=-1)
m.fit(all_features, all_y)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [66]:
pred = m.predict(consum_gr_test)

In [68]:
pred.mean()

0.1826377412028501

In [94]:
consum_gr_test['cell_encoding'] = pred

In [71]:
consum_gr_test.reset_index(inplace=True)

In [98]:
cell2targ_gr_test = consum_gr_test[['CELL_LAC_ID', 'cell_encoding']].copy()

### Aggregation

#### Aggregate features

In [77]:
for df in (raw_train, raw_test):
    df.drop('SNAP_DATE', axis=1, inplace=True)

In [78]:
for df in (raw_train,raw_test):
    print('====')
    for n, c in df.iteritems():
        if c.nunique() < 10:
            print(n, '-->', c.nunique(), c.dtype)

====
COM_CAT#1 --> 8 int64
BASE_TYPE --> 2 int64
ACT --> 2 int64
COM_CAT#7 --> 9 int64
DEVICE_TYPE_ID --> 8 float64
INTERNET_TYPE_ID --> 3 float64
COM_CAT#24 --> 1 object
COM_CAT#25 --> 1 int64
COM_CAT#26 --> 2 int64
====
COM_CAT#1 --> 8 int64
BASE_TYPE --> 2 int64
ACT --> 2 int64
COM_CAT#7 --> 9 int64
DEVICE_TYPE_ID --> 8 float64
INTERNET_TYPE_ID --> 3 float64
COM_CAT#24 --> 7 object
COM_CAT#25 --> 2 int64
COM_CAT#26 --> 2 int64


In [79]:
train_cats(raw_train)
apply_cats(raw_test, raw_train)

In [80]:
tr_gr = raw_train.groupby('SK_ID', as_index=False).agg(['mean', 'max', 'min'])
test_gr = raw_test.groupby('SK_ID', as_index=False).agg(['mean','max', 'min'])

In [81]:
for df in (tr_gr, test_gr):
    df.columns = ['_'.join(c) for c in df.columns]

In [82]:
# add contact date and labels
tr_gr = pd.merge(tr_gr, train_ids, on='SK_ID')

In [83]:
test_gr = pd.merge(test_ids, test_gr, on='SK_ID', how='left')

In [84]:
for df in (tr_gr, test_gr):
    df['CONTACT_DATE'] = pd.to_datetime(df['CONTACT_DATE'], format='%d.%m').dt.day

In [85]:
(tr_gr.shape,test_gr.shape)

((3976, 102), (5221, 101))

####  Aggregation consumption

In [101]:
for df in (raw_consum_train, raw_consum_test):
    df.drop(['MON'], axis=1, inplace=True)

In [102]:
for df in (raw_consum_train, raw_consum_test):
    print('====')
    for n, c in df.iteritems():
        if c.nunique() < 10:
            print(n, '-->', c.nunique(), c.dtype)

====
====


In [103]:
raw_consum_train = pd.merge(raw_consum_train, cell2targ_gr, on='CELL_LAC_ID', how='left')
raw_consum_test = pd.merge(raw_consum_test, cell2targ_gr_test, on='CELL_LAC_ID', how='left')

In [118]:
raw_consum_test.head()

,SK_ID,CELL_LAC_ID,SUM_MINUTES,SUM_DATA_MB,SUM_DATA_MIN,cell_encoding
0,6357,104453,0.000000,0.000000e+00,0.000234,0.181234
1,6357,8217,0.000000,2.200000e-07,0.001167,0.230895
2,6357,412480,0.002722,5.590000e-06,0.010187,0.190651
3,6357,355470,0.000000,5.309000e-05,0.002480,0.121937
4,6357,45770,0.000000,4.000000e-08,0.000084,0.243234


In [108]:
aggs = {'SUM_MINUTES':['mean', 'max', 'min'],
        'SUM_DATA_MB':['mean', 'max', 'min'],
        'SUM_DATA_MIN':['mean', 'max', 'min'],
        'cell_encoding': 'mean'}

In [123]:
consum_gr_tr = raw_consum_train.groupby('SK_ID').agg(aggs)
consum_gr_test = raw_consum_test.groupby('SK_ID').agg(aggs)

In [125]:
for df in (consum_gr_tr, consum_gr_test):
    df.columns = ['_'.join(c) for c in df.columns]

In [126]:
consum_gr_tr.head()

,SUM_MINUTES_mean,SUM_MINUTES_max,SUM_MINUTES_min,SUM_DATA_MB_mean,SUM_DATA_MB_max,SUM_DATA_MB_min,SUM_DATA_MIN_mean,SUM_DATA_MIN_max,SUM_DATA_MIN_min,cell_encoding_mean
SK_ID,,,,,,,,,,
2,0.000554,0.017112,0.000000,0.000047,0.001820,0.0,0.004205,0.225086,0.0,0.160109
3,0.004911,0.027122,0.000000,0.000398,0.003146,0.0,0.060658,0.377977,0.0,0.020704
4,0.005607,0.098539,0.000003,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.231425
5,0.000842,0.029620,0.000000,0.000871,0.062863,0.0,0.004767,0.184683,0.0,0.041759
6,0.001875,0.017714,0.000029,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.440630


#### Merge features and consumptions

In [127]:
# add contact date and labels
tr_gr = pd.merge(tr_gr, consum_gr_tr, on='SK_ID')

In [128]:
test_gr = pd.merge(test_gr, consum_gr_test, on='SK_ID', how='left')

#### Add foldID to train 

In [133]:
tr_gr['foldID'] = np.random.randint(0,5+1,len(tr_gr))

In [134]:
(tr_gr.shape,test_gr.shape)

((3969, 113), (5221, 111))

### Train

In [135]:
tr_gr = tr_gr.sample(frac=1).reset_index(drop=True)

In [136]:
df, y, nas = proc_df(tr_gr, 'CSI', ignore_flds=['SK_ID', 'foldID'])

In [137]:
test, _, _ = proc_df(test_gr, ignore_flds=['SK_ID'], na_dict=nas)

In [138]:
test.shape, df.shape

((5221, 126), (3969, 127))

In [154]:
auc_scores = []
for i in range(5):
    # create ids for train and validation
    idx_train = df.loc[df.foldID != i].index
    idx_val = df.loc[df.foldID == i].index
    
    
    # make train and validation set
    all_features = df.drop(['SK_ID', 'foldID'], axis=1).copy()
    X_train, X_valid = all_features.loc[idx_train], all_features.loc[idx_val]
    y_train, y_valid = y[idx_train], y[idx_val]
    
    m = RandomForestClassifier(n_jobs=-1, n_estimators=10, min_samples_leaf=50,
                               max_features=0.5)
    m.fit(X_train, y_train)
    
    y_pred = m.predict_proba(X_valid)[:,1]
    auc_scores.append(metrics.roc_auc_score(y_valid, y_pred))
    
print(auc_scores)
    

[0.9961872586872588, 0.9951709178422711, 0.9959286803966438, 0.9961163227016886, 0.9972194135490395]


### Predict on test and submit

In [155]:
m = RandomForestClassifier(n_jobs=-1, n_estimators=10, min_samples_leaf=50,
                               max_features=0.5)
m.fit(df.drop(['SK_ID', 'foldID'], axis=1), y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features=0.5, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=50, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [156]:
pred_y = m.predict_proba(test.drop(['SK_ID'], axis=1))[:,1]

In [160]:
(pred_y> 0.1).sum()

244

In [161]:
subm = pd.DataFrame(pred_y)

In [162]:
subm.shape

(5221, 1)

In [163]:
subm.to_csv('submits/rf_targ_enc_for_cells_2lvl.csv', index=False, header=False)